Hi, I hope you're doing well. Today, we'll compare two approaches to outlier removal **:**<br><br> 1. **Column-wise removal:** Find and drop outliers for each column sequentially (first column, then second column, then third, and so on).<br>2. **Dataset-level removal:** First identify all outliers across the entire dataset, then remove them simultaneously.<br><br>Let's get started.<br>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('/content/Housing.csv')
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [ ]:
df.describe()

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


In [ ]:
df.isnull().sum()

,0
price,0
area,0
bedrooms,0
bathrooms,0
stories,0
mainroad,0
guestroom,0
basement,0
hotwaterheating,0
airconditioning,0


In [ ]:
df.duplicated().sum()

np.int64(0)

# **First Approach...**

In [ ]:
numericColumnsName=df.select_dtypes(include=np.number).columns
numericColumnsName

Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking'], dtype='object')

In [ ]:
#number of outliers for each column
for x in numericColumnsName:
  Q2=df[x].median()
  Q1=df[x].quantile(0.25)
  Q3=df[x].quantile(0.75)
  IQR=Q3-Q1
  lower_bound=Q1-1.5*IQR
  upper_bound=Q3+1.5*IQR
  numberOfOutlier=df[(df[x]>upper_bound) | (df[x]<lower_bound)][x].count()
  print(f'--->number of outlier in {x} column is {numberOfOutlier}')
  #print(df[(df[x]>upper_bound) | (df[x]<lower_bound)])
  #print('-----------------------------------')

--->number of outlier in price column is 15
--->number of outlier in area column is 12
--->number of outlier in bedrooms column is 12
--->number of outlier in bathrooms column is 1
--->number of outlier in stories column is 41
--->number of outlier in parking column is 12


In [ ]:
#removing outlier
df1=df.copy()
for x in numericColumnsName:
  Q1=df1[x].quantile(0.25)
  Q3=df1[x].quantile(0.75)
  IQR=Q3-Q1
  lower_bound=Q1-1.5*IQR
  upper_bound=Q3+1.5*IQR
  df1=df1[(df1[x]>=lower_bound) & (df1[x]<=upper_bound)]
df1.shape



(365, 13)

*** But shape of new df1 is eye catching small from df. Because, the sum of all **nonunique** outlier for all columns is **93**. So,the shape of df1 be minimum **(452,13)**. But, this is (365, 13). Therefore, here must be a wrong.


# **FINDING THE ERROR OF THE APPROACH**

In [ ]:
#lower & upper bound before droping outliers of "price"
df1=df.copy()
for each in numericColumnsName:
  q1=df1[each].quantile(0.25)
  q3=df1[each].quantile(0.75)
  iqr=q3-q1
  upperLimit=q3+(1.5*iqr)
  lowerLimit=q1-(1.5*iqr)
  print(f'{each}-->UpperLimit:{upperLimit}, LowerLimit:{lowerLimit}')

price-->UpperLimit:9205000.0, LowerLimit:-35000.0
area-->UpperLimit:10500.0, LowerLimit:-540.0
bedrooms-->UpperLimit:4.5, LowerLimit:0.5
bathrooms-->UpperLimit:3.5, LowerLimit:-0.5
stories-->UpperLimit:3.5, LowerLimit:-0.5
parking-->UpperLimit:2.5, LowerLimit:-1.5


In [ ]:
#remove outlier of 'price' columns
Q1=df1['price'].quantile(0.25)
Q3=df1['price'].quantile(0.75)
IQR=Q3-Q1
lower_bound=Q1-1.5*IQR
upper_bound=Q3+1.5*IQR
withoutOutlierOfPrice=df1[(df1['price']>=lower_bound) & (df1['price']<=upper_bound)]
withoutOutlierOfPrice.shape

(530, 13)

In [ ]:
#lower & upper bound after droping outliers of "price"
for each in numericColumnsName:
  q1=withoutOutlierOfPrice[each].quantile(0.25)
  q3=withoutOutlierOfPrice[each].quantile(0.75)
  iqr=q3-q1
  upperLimit=q3+(1.5*iqr)
  lowerLimit=q1-(1.5*iqr)
  print(f'{each}-->UpperLimit:{upperLimit}, LowerLimit:{lowerLimit}')

price-->UpperLimit:8855000.0, LowerLimit:175000.0
area-->UpperLimit:10468.125, LowerLimit:-604.875
bedrooms-->UpperLimit:4.5, LowerLimit:0.5
bathrooms-->UpperLimit:1.0, LowerLimit:1.0
stories-->UpperLimit:3.5, LowerLimit:-0.5
parking-->UpperLimit:2.5, LowerLimit:-1.5


**Droping the outlier of one column can affect on the value of lower_bound, upper_bound of other columns.** So, although a value is not outlier for a columns with respect to the main df, it maybe outlier for modified df. Therefore, the above problem is happened. So, droping outlier by the above method is not a good approach.

# **Second Approach...**

In [ ]:
dfOutiers=pd.DataFrame(None)
for x in numericColumnsName:
  Q2=df[x].median()
  Q1=df[x].quantile(0.25)
  Q3=df[x].quantile(0.75)
  IQR=Q3-Q1
  lower_bound=Q1-1.5*IQR
  upper_bound=Q3+1.5*IQR
  numberOfOutlier=df[(df[x]>upper_bound) | (df[x]<lower_bound)][x].count()
  temp=df[(df[x]>upper_bound) | (df[x]<lower_bound)]

  dfOutiers=pd.concat([dfOutiers,temp],axis=0)#collect all outlier of all columns

  print(f'--->number of outlier in {x} column is {numberOfOutlier}')
  #print(df[(df[x]>upper_bound) | (df[x]<lower_bound)])
  #print('-----------------------------------')

--->number of outlier in price column is 15
--->number of outlier in area column is 12
--->number of outlier in bedrooms column is 12
--->number of outlier in bathrooms column is 1
--->number of outlier in stories column is 41
--->number of outlier in parking column is 12


In [ ]:
dfOutiers.shape

(93, 13)

In [ ]:
dfOutiers.duplicated().sum()

np.int64(11)

In [ ]:
dfOutiers.drop_duplicates(inplace=True)

In [ ]:
dfOutiers.duplicated().sum()

np.int64(0)

In [ ]:
dfOutiers.shape

(82, 13)

In [ ]:
dfOutiers.index


Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  56,  64,  66,  69, 125, 129, 186, 191, 211, 403,  28,  34,  89,
       112, 143, 152, 271, 340, 356, 395, 536,  17,  26,  30,  31,  35,  37,
        38,  39,  41,  42,  43,  44,  46,  47,  50,  51,  52,  53,  57,  58,
        59,  71,  72,  73,  83,  92,  94, 102, 105, 124, 131, 135, 140, 145,
       160, 220, 226, 247,  93, 225, 299, 304, 323, 331, 401, 472],
      dtype='int64')

In [ ]:
for x in dfOutiers.index:
  df.drop(index=x,inplace=True, axis=0)

In [ ]:
df.shape

(463, 13)

So, by above discussion, we can say that, **the second approach is better than the first approach**
<br>